In [1]:
import os
import numpy as np
import pandas as pd
from time import process_time
from itertools import product
from multiprocess import Pool
import threading
from dust_particle_integration import integration

In [2]:
# input variables

Degree_To_Rad = np.pi/180.
yr = 365*24*3600 # yr in [s]

k1_array = np.linspace(0.9, 1.1, 2) # m_star/m_Sun
k2_array = np.linspace(0.9, 1.1, 2) # m_planet/m_Jupiter
k_ap_array = np.array([10]) # a_Planet/R_Sun
beta_array = np.linspace(0.14, 0.16, 2) # F_rad/F_grav
k_init = np.array([3.]) # a_initial/a_Planet
e_initial = np.array([0.]) # initial dust particle eccentricity (=0)
inc_initial = np.array([0.*Degree_To_Rad]) # initial dust particle eccentricity (=0)
endtime = np.array([100 * yr]) # integration time

N_ = 10 # number of particles
N_array = np.full(N_, 1) 

input_array = []
r = product(k1_array, k2_array, k_ap_array, beta_array, k_init, e_initial, inc_initial, endtime, N_array)
for ri in r:
    input_array.append(ri)


In [3]:
from tabulate import tabulate

text = """
All subprocesses done.
"""

output = tabulate([[text]], tablefmt='grid')

In [4]:
# create an empty dataframe
paralabels = ["m_Star/m_Sun", "m_Planet/m_J", "a_p/R_Sun", "R_sub/R_Sun", "R_Planet/R_Sun",
                  "beta", "a_d_i/a_p", "e_d_i", "inc_d_i", "Omega_d_i", "pomega_d_i", "f_d_i",  
                  "final_fate", "lifetime[yr]", "x_p_f_xyz[m]", "v_p_f_xyz[m/s]", "x_d_f_xyz[m]",
                  "v_d_f_xyz[m/s]", "CJ_final", "a_d_f", "e_d_f", "vrel_f[km/s]", "mu", "CJ", "a_d", "e_d" ]
df_para = pd.DataFrame(columns=paralabels)

In [5]:
%%time

if __name__ == "__main__":
                
    pool = Pool(processes=128) # pool number = cpu number by default

    # pool_outputs = pool.map(integration, input_array)
    for pool_output in pool.imap(integration, input_array):
        # merge data
        df_para = pd.concat([df_para, pool_output], axis=0)
        df_para = df_para.reset_index(drop = True)
        # print ('index of data:', len(df_para))
        
    # save data
    df_para = df_para.sort_values(by=['m_Star/m_Sun', 'm_Planet/m_J', 'a_p/R_Sun'], ascending = False)
    df_para.to_csv('./test.csv', header=True, index=False)
    
    pool.close() # close() doesn't kill any process; it just closes a pipe which informs that there will be no more data coming through it.
    pool.join() # Killed processes send a signal informing their parents that they are quite dead.
    
    print (output)


Sublimation: c_uint(2) at 82.52738Sublimation:  [yr]
c_uint(2) at 81.18541 [yr]No dusts left. Finish integration. :)

Sublimation: c_uint(2) at 82.42744 [yr]No dusts left. Finish integration. :)

Sublimation: c_uint(2) at 82.41419 [yr]
No dusts left. Finish integration. :)
No dusts left. Finish integration. :)
Sublimation: c_uint(2) at 82.58552 [yr]
No dusts left. Finish integration. :)
Sublimation: c_uint(2) at 81.54642 [yr]
No dusts left. Finish integration. :)
Hit the Planet: c_uint(2) at 85.07246 [yr]
No dusts left. Finish integration. :)
Sublimation: c_uint(2) at 84.39136 [yr]
No dusts left. Finish integration. :)
Sublimation: c_uint(2) at 86.19454 [yr]
No dusts left. Finish integration. :)
Sublimation: c_uint(2) at 89.59961 [yr]
No dusts left. Finish integration. :)
Sublimation: c_uint(2) Hit the Planet:at 91.19855 c_uint(2) [yr] 
at 89.59676 [yr]
No dusts left. Finish integration. :)
Sublimation: c_uint(2)No dusts left. Finish integration. :) 
at 90.79466 [yr]
No dusts left. Fin

In [6]:
df_para_rd = pd.read_csv('./test.csv', on_bad_lines='skip')
df_para = df_para_rd.sort_values(by=['m_Star/m_Sun', 'm_Planet/m_J', 'a_p/R_Sun'], ascending = False)
df_para

,m_Star/m_Sun,m_Planet/m_J,a_p/R_Sun,R_sub/R_Sun,R_Planet/R_Sun,beta,a_d_i/a_p,e_d_i,inc_d_i,Omega_d_i,...,x_d_f_xyz[m],v_d_f_xyz[m/s],CJ_final,a_d_f,e_d_f,vrel_f[km/s],mu,CJ,a_d,e_d
0,1.1,1.1,10,7.507092,0.102372,0.14,3.0,0.0,0.0,3.997764,...,"[2252833672.197864, 13288967445.55253, 0.0]","[-68157.8457591071, 46314.213124311835, 0.0]",-2.567689,1.057305e+10,0.485004,NaN,NaN,[-3.49906785 -3.48582717 -3.4724281 -3.458866...,[1.82828868e+10 1.81948822e+10 1.79146198e+10 ...,[0.14133021 0.13596575 0.14097267 0.14181243 0...
1,1.1,1.1,10,7.507092,0.102372,0.14,3.0,0.0,0.0,1.707763,...,"[8234160754.969866, 11901739416.752453, 0.0]","[-42733.55977975122, 61019.7602499724, 0.0]",-2.567677,1.061075e+10,0.486786,NaN,NaN,[-3.49907664 -3.48583606 -3.47243711 -3.458875...,[1.82759718e+10 1.81939525e+10 1.79075985e+10 ...,[0.14171847 0.13601644 0.14138069 0.14208788 0...
2,1.1,1.1,10,7.507092,0.102372,0.14,3.0,0.0,0.0,3.091761,...,"[8275819524.117904, -13373477710.970531, 0.0]","[49428.582792091736, 42566.842208516304, 0.0]",-2.567627,1.072256e+10,0.486495,NaN,NaN,[-3.49908977 -3.4858493 -3.47245055 -3.458888...,[1.82667190e+10 1.81526819e+10 1.78969795e+10 ...,[0.14223808 0.13832164 0.14200403 0.14162702 0...
3,1.1,1.1,10,7.507092,0.102372,0.14,3.0,0.0,0.0,2.898955,...,"[-6232573796.591253, 3408748848.0702477, 0.0]","[-118618.92396573426, -97097.59709861378, 0.0]",-2.567715,1.057942e+10,0.483013,NaN,NaN,[-3.49904339 -3.48580245 -3.47240307 -3.458840...,[1.83304277e+10 1.81767288e+10 1.79618318e+10 ...,[0.1386626 0.13697378 0.13830158 0.13939779 0...
4,1.1,1.1,10,7.507092,0.102372,0.14,3.0,0.0,0.0,1.281598,...,"[12491200157.642889, -9830714063.020443, 0.0]","[35719.408124495894, 53012.23336940521, 0.0]",-2.567636,1.071664e+10,0.487356,NaN,NaN,[-3.49909706 -3.48585668 -3.47245801 -3.458896...,[1.82619605e+10 1.81705994e+10 1.78928514e+10 ...,[0.14250536 0.13731535 0.14224827 0.14218125 0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.9,0.9,10,5.555798,0.103197,0.16,3.0,0.0,0.0,3.887901,...,"[8680791251.31434, 9996528923.290829, 0.0]","[-69921.50251751482, 22373.941701812266, 0.0]",-2.528031,1.029200e+10,0.473817,NaN,NaN,[-3.45494971 -3.44268762 -3.430288 -3.417747...,[1.80572774e+10 1.78029256e+10 1.77206167e+10 ...,[0.15616127 0.16107778 0.15595048 0.15590662 0...
76,0.9,0.9,10,5.555798,0.103197,0.16,3.0,0.0,0.0,1.075713,...,"[5362261972.447012, -715613426.6029725, 0.0]","[28643.428329520444, 162919.283621544, 0.0]",-2.528103,1.035610e+10,0.478561,NaN,NaN,[-3.45484116 -3.44257778 -3.43017699 -3.417634...,[1.79663853e+10 1.78491564e+10 1.76117991e+10 ...,[0.16142395 0.15841946 0.16218285 0.16094831 0...
77,0.9,0.9,10,5.555798,0.103197,0.16,3.0,0.0,0.0,2.028968,...,"[10002674066.080389, -2359962741.0716114, 0.0]","[66897.18939676936, 74062.92137998564, 0.0]",-2.528185,1.049669e+10,0.481668,NaN,NaN,[-3.45483224 -3.44256876 -3.43016787 -3.417625...,[1.80271443e+10 1.77913712e+10 1.76582135e+10 ...,[0.15790461 0.16178791 0.15937012 0.15752248 0...
78,0.9,0.9,10,5.555798,0.103197,0.16,3.0,0.0,0.0,5.397609,...,"[4673982199.576148, 13181818922.671572, 0.0]","[-67606.52602260679, 422.8367800079045, 0.0]",-2.528044,1.027520e+10,0.473101,NaN,NaN,[-3.45492534 -3.44266298 -3.43026308 -3.417721...,[1.80537016e+10 1.78090858e+10 1.77205133e+10 ...,[0.15636808 0.16072004 0.15595455 0.15608185 0...


In [7]:
print(len(df_para[df_para['final_fate']=='collision']))
print(len(df_para[df_para['final_fate']=='sublimation']))
print(len(df_para[df_para['final_fate']=='escape']))
print(len(df_para[df_para['final_fate']=='incomplete']))

8
18
0
54


In [8]:
df_para[df_para['final_fate']=='collision']

,m_Star/m_Sun,m_Planet/m_J,a_p/R_Sun,R_sub/R_Sun,R_Planet/R_Sun,beta,a_d_i/a_p,e_d_i,inc_d_i,Omega_d_i,...,x_d_f_xyz[m],v_d_f_xyz[m/s],CJ_final,a_d_f,e_d_f,vrel_f[km/s],mu,CJ,a_d,e_d
10,1.1,1.1,10,7.507092,0.102372,0.16,3.0,0.0,0.0,0.144803,...,"[6013850314.381078, 3349282431.4133997, 0.0]","[2271.6218589799846, 170146.54750886193, 0.0]",-2.527803,1.852274e+10,0.737894,84.996586,0.995637,[-3.45490879 -3.4399026 -3.42468977 -3.409263...,[1.80508633e+10 1.77968043e+10 1.75866416e+10 ...,[0.15653224 0.15905624 0.15929935 0.16168939 0...
12,1.1,1.1,10,7.507092,0.102372,0.16,3.0,0.0,0.0,5.924022,...,"[-1836962975.8987033, -6634472292.225899, 0.0]","[122057.21171435805, -121499.00875446292, 0.0]",-2.518314,2.073981e+10,0.761036,84.151066,0.909081,[-3.45495828 -3.43995279 -3.42474065 -3.409315...,[1.80583985e+10 1.78133203e+10 1.75710833e+10 ...,[0.15609643 0.15809365 0.16020438 0.16202789 0...
18,1.1,1.1,10,7.507092,0.102372,0.16,3.0,0.0,0.0,3.082148,...,"[-6886085274.88761, 8980155.8655084, 0.0]","[-81082.00756918623, -166886.90927137047, 0.0]",-2.527068,1.079661e+11,0.948599,84.377145,0.992392,[-3.45483495 -3.43982774 -3.42461399 -3.409186...,[1.79717323e+10 1.78165089e+10 1.75738417e+10 ...,[0.161114 0.15779076 0.16003997 0.15652409 0...
19,1.1,1.1,10,7.507092,0.102372,0.16,3.0,0.0,0.0,1.551912,...,"[-6899196741.125084, 112165126.2425921, 0.0]","[-85213.43870114161, -155006.9691046819, 0.0]",-2.527846,2.918796e+10,0.820841,84.525484,0.794970,[-3.4548494 -3.43984239 -3.42462873 -3.409201...,[1.80354115e+10 1.77751871e+10 1.76072247e+10 ...,[0.15742618 0.16029248 0.15809548 0.16085424 0...
23,1.1,0.9,10,7.507092,0.103197,0.14,3.0,0.0,0.0,0.561863,...,"[6629609042.895896, 1862057660.0761168, 0.0]","[36274.8011300053, 166868.0452832252, 0.0]",-2.568237,1.728856e+10,0.708345,80.602407,0.987910,[-3.49903402 -3.48579417 -3.47239592 -3.458834...,[1.83285826e+10 1.81642927e+10 1.79630884e+10 ...,[0.13877816 0.13768356 0.138334 0.13908558 0...
24,1.1,0.9,10,7.507092,0.103197,0.14,3.0,0.0,0.0,4.679909,...,"[-2430817447.5445933, 6587643559.619207, 0.0]","[-102493.33309677333, -122969.91333603664, 0.0]",-2.567975,1.243928e+10,0.625245,80.052829,0.992791,[-3.49904437 -3.48580462 -3.47240651 -3.458845...,[1.82950834e+10 1.81811165e+10 1.79298969e+10 ...,[0.14065801 0.13674949 0.14019825 0.14091051 0...
25,1.1,0.9,10,7.507092,0.103197,0.14,3.0,0.0,0.0,5.336388,...,"[-4309832749.291215, 5542262880.5903015, 0.0]","[-77188.30771520604, -160879.9931648569, 0.0]",-2.563005,3.232406e+10,0.830402,80.677460,0.999557,[-3.49916604 -3.48592774 -3.47253109 -3.458971...,[1.83672119e+10 1.81099911e+10 1.79966727e+10 ...,[0.13661202 0.14072162 0.13656226 0.13667725 0...
34,1.1,0.9,10,7.507092,0.103197,0.16,3.0,0.0,0.0,5.368968,...,"[-6886553147.537909, -71914667.22187012, 0.0]","[-78589.75669455278, -147961.82014629504, 0.0]",-2.521939,1.634532e+10,0.697516,80.753471,0.929283,[-3.45483558 -3.43982991 -3.42461772 -3.409192...,[1.79623193e+10 1.77541279e+10 1.76272384e+10 ...,[0.16167258 0.1615021 0.15692936 0.15681708 0...
